# Interactive simulation tutorial part 1: RSA

This tutorial is intended to give an introduction to Rational Speech Act (RSA) agents within the Lanag simulation framework. It is part of the Supplementary Information for *submitted*, which we will refer to as the *main paper*.

## 1. Loading libraries
First we need to load some libraries.

In [1]:
import $ivy.`com.markblokpoel::lanag-core:0.3.2`
interp.load.cp(new java.net.URL("https://github.com/markblokpoel/lanag-ambiguityhelps/blob/jupyternotebooks/binaries/lanag-ambiguityhelps-0.1.jar?raw=true"))

import $ivy.$                                   


In [2]:
import com.markblokpoel.lanag.rsa.Lexicon
import com.markblokpoel.lanag.ambiguityhelps.{RSA1ShotAgent, RSA1ShotInteraction}

import com.markblokpoel.lanag.rsa.Lexicon

import com.markblokpoel.lanag.ambiguityhelps.{RSA1ShotAgent, RSA1ShotInteraction}

### 1.1 Helper functions
These are functions I may want to add to the lanag-core library.

In [3]:
def lex2md(l: Lexicon): String = {
    val vs = l.vocabularySize
    val cs = l.contextSize
    
    var output = ""
    for(v <- -1 until vs) {
        for(c <- -1 until cs) {
            if(c < 0 && v < 0) output += "|&nbsp;|"
            else if (c < 0) output += "|$S_"+v+"$|"
            else if (v < 0) {
                output += "$C_"+c+"$|"
                if(c==cs-1) output += "\n"+ ((0 until 3).map(_ => "|--").mkString+"|")
            } else output += l(v,c)+"|"
        }
        output += "\n"
    }
    output
}

case object ReferentialIntention {
  def apply(content: Int): com.markblokpoel.lanag.core.ReferentialIntention = com.markblokpoel.lanag.core.ReferentialIntention(Some(content))
}

defined function lex2md
defined object ReferentialIntention

## 2. RSA agent

In the Rational Speech Act model, an agent’s lexicon is defined by a (possibly) graded mapping between a set of signals and a set of referents. This mapping is encoded in a matrix, where each column represents a referent, and each row represents a signal. An agent is defined by its matrix and its order of pragmatic inference.

In this tutorial we will follow the example from the main paper *submitted* about two friends (`friend1` and `friend2`) talking about two classmates whos names they may not know. The snippet below illustrates how a zero order agent can be defined. This agent has a vocabulary $V=\{\text{red hair},\text{tall one},\text{gamer}\}$ and a referent set $R=\{\text{Nora},\text{Asla}\}$. You will notice that the labels of the signals and referents are not present in the simulation code, as these are for our current purpose not important.

| &nbsp; |  Nora  |  Asla |
|---|--------|-------|
| *red hair* | 1 | 0 |
| *tall one* | 0 | 1 |
| *gamer*    | 1 | 1 |

In [4]:
val mapping = Vector[Vector[Double]](
  Vector(1,0),
  Vector(0,1),
  Vector(1,1)
)
val lexicon = Lexicon(mapping)
val friend1 = new RSA1ShotAgent(lexicon, order = 0)

mapping: Vector[Vector[Double]] = Vector(
  Vector(1.0, 0.0),
  Vector(0.0, 1.0),
  Vector(1.0, 1.0)
)
lexicon: Lexicon = Lexicon(3, 2, Vector(1.0, 0.0, 0.0, 1.0, 1.0, 1.0))
friend1: RSA1ShotAgent = Agent with order 0
1.0	0.0	
0.0	1.0	
1.0	1.0	


In [5]:
Markdown(s"""
**Viewing a lexicon**

`friend 1`'s lexicon can be viewed as a table.

${lex2md(friend1.originalLexicon)}
""")


**Viewing a lexicon**

`friend 1`'s lexicon can be viewed as a table.

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|1.0|0.0|
|$S_1$|0.0|1.0|
|$S_2$|1.0|1.0|



To get the agent to engage its pragmatic reasoning capacity, we need to tell the simulation framework if the agent is a speaker or a listener. This is done by calling the function `.asSpeaker` or `.asListener` on the object which return a new object where the agent is a speaker or listener. This will internally perform pragmatic inference (based on RSA, see Formulas~1-6 in the main paper) at the specified order to compute a conditional probability mapping. This mapping can be accessed with `.inferredLexicon`.

*If you are running this notebook interactively, you can play around with the map and the order to see how this changes the conditional probability mapping. For example, try setting order to 1 and then 2 or change the agent’s lexicon.*

In [6]:
val friend1AsListener = friend1.asListener
val friend1AsSpeaker = friend1.asSpeaker

friend1AsListener: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotListener = RSA1ShotListener(
  Lexicon(3, 2, Vector(1.0, 0.0, 0.0, 1.0, 1.0, 1.0)),
  0
)
friend1AsSpeaker: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotSpeaker = RSA1ShotSpeaker(
  Lexicon(3, 2, Vector(1.0, 0.0, 0.0, 1.0, 1.0, 1.0)),
  0
)

## 2.1 Zero order speaker and communicator

`friend1`'s zero order inferred lexicon consists of
probability distributions for signals over referents. For order zero this
is computed by normalizing over rows.

In [7]:
Markdown(s"${lex2md(friend1AsListener.inferredLexicon)}")

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|1.0|0.0|
|$S_1$|0.0|1.0|
|$S_2$|0.5|0.5|


For speakers, the inferred lexicon consists of distributions over signals. It is computed by normalizing over columns. Here is `friend§`'s inferred lexicon at order zero.

In [8]:
Markdown(s"${lex2md(friend1AsSpeaker.inferredLexicon)}")

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|0.5|0.0|
|$S_1$|0.0|0.5|
|$S_2$|0.5|0.5|


## 2.2 Producing a signal
Let's make `friend1` communicate a selected intention using `.produceSignal(.)`. This function returns two objects, the selected signal and some data. We will ignore the data for now. In a moment, we will create a friend that will try to intepret the signal.

In [9]:
val intention = 0
val speaker = friend1.asSpeaker
val refIntention = ReferentialIntention(intention)
val (signal, data) = speaker.produceSignal(refIntention)

intention: Int = 0
speaker: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotSpeaker = RSA1ShotSpeaker(
  Lexicon(3, 2, Vector(1.0, 0.0, 0.0, 1.0, 1.0, 1.0)),
  0
)
refIntention: com.markblokpoel.lanag.core.ReferentialIntention = ReferentialIntention(
  Some(0)
)
signal: com.markblokpoel.lanag.core.ContentSignal = ContentSignal(Some(2))
data: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotDataStructures.SpeakerData = SpeakerData(Some(1.0))

In [10]:
Markdown(s"`friend1` communicates referent $$R_$intention$$ using signal $$S_${signal.content.get}$$.")

`friend1` communicates referent $R_0$ using signal $S_2$.

### 2.2.1 How `produceSignal` works
The function `produceSignal` does two things. First, it transforms an intention into a distribution over intentions $Pr(R)$, where the intention has probability 1. Next, it computes a posterior distribution over signals given a referent by computing the dot product between the distribution and the $n^\text{th}$-order inferred lexicon. Then it selects a signal using the softmax function. Here, we set beta to infinite, always selecting the signal with the highest probability (or at random if multiple signals have highest probability).

In [11]:
import com.markblokpoel.lanag.math.Probability.softArgMax
val intentionDistribution = Vector.tabulate(speaker.originalLexicon.contextSize)(n => if (n == intention) 1.0 else 0.0)
val posteriorDistribution = speaker.inferredLexicon dot intentionDistribution
softArgMax(posteriorDistribution, beta = Double.PositiveInfinity)

import com.markblokpoel.lanag.math.Probability.softArgMax

intentionDistribution: Vector[Double] = Vector(1.0, 0.0)
posteriorDistribution: Vector[Double] = Vector(0.5, 0.0, 0.5)
res10_3: Option[Int] = Some(0)

### 2.2.2 Exploring speaker behaviour
*This section assumes you are running this notebook interactively.*

The lines of code below are all we need to explore `friend1`'s behaviour as a speaker. You can change the order of pragmatic reasoning and try different intentions to explore how they affect the speaker's inferred lexicon and produced signals.

> You may have noticed objects `Option`, `Some` and `None` pop up in the code or results. This construct is used when a function may not be able to return a value. For example, calling `softArgMax` on an empty distribution will result in a `None`. To 'get' a value from an option-wrapped value, you can use `.get` or better `.getOrElse(x)`, where x is the returned value if called on `None`.


Here are some questions to guide your exploration of speaker behaviour:

1. What signal has the highest probability given that intention 0?

2. If you run the same intention multiple times, does the outcome change and why?

3. As a zero order agent, `friend1` cannot reason about the ambiguity in her lexicon and hence when multiple signal options are valid, she selects a signal at random from them. Try setting the order to 1 and observe how both the inferred lexicon and selected signals change.

5. With `friend1` as a higher order speaker, try running the code multiple times with the same intention. Observe that for a higher order speaker with this particular lexicon, there is a clear better signal.


In [12]:
val speaker = friend1.withOrder(0).asSpeaker
val (signal, data) = speaker.produceSignal(ReferentialIntention(0))

Markdown(s"""
**First production results**

${lex2md(speaker.inferredLexicon)} 

The selected signal is $$S_${signal.content.get}$$.
""")


**First production results**

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|0.5|0.0|
|$S_1$|0.0|0.5|
|$S_2$|0.5|0.5|
 

The selected signal is $S_2$.


speaker: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotSpeaker = RSA1ShotSpeaker(
  Lexicon(3, 2, Vector(1.0, 0.0, 0.0, 1.0, 1.0, 1.0)),
  0
)
signal: com.markblokpoel.lanag.core.ContentSignal = ContentSignal(Some(2))
data: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotDataStructures.SpeakerData = SpeakerData(Some(1.0))

## 3. A second agent
Let's create a partner for our friend creatively called `friend2`. For now, we give this agent the same lexicon as `friend1`, but it could be different.

In [13]:
val friend2 = new RSA1ShotAgent(friend1.originalLexicon, order = 0)

friend2: RSA1ShotAgent = Agent with order 0
1.0	0.0	
0.0	1.0	
1.0	1.0	


We will make `friend2` the listener of the two. Let’s see how `friend2` would interpret a signal generated by `friend1`.

In [14]:
val listener = friend2.withOrder(0).asListener
val speaker = friend1.withOrder(0).asSpeaker
val intention = 0
val (signal,_) = speaker.produceSignal(ReferentialIntention(intention))
val (inferredIntention,_) = listener.interpretSignal(signal)

Markdown(s"""
**First communication result**

`friend1` communicates referent $$R_$intention$$ using signal $$S_${signal.content.get}$$.

`friend2` infers referent $$R_${inferredIntention.content.get}$$.

**The agent's inferred lexicons**

`friend1`*'s lexicon as speaker*

${lex2md(speaker.inferredLexicon)}

`friend2`*'s lexicon as listener*

${lex2md(listener.inferredLexicon)}
""")


**First communication result**

`friend1` communicates referent $R_0$ using signal $S_0$.

`friend2` infers referent $R_0$.

**The agent's inferred lexicons**

`friend1`*'s lexicon as speaker*

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|0.5|0.0|
|$S_1$|0.0|0.5|
|$S_2$|0.5|0.5|


`friend2`*'s lexicon as listener*

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|1.0|0.0|
|$S_1$|0.0|1.0|
|$S_2$|0.5|0.5|



listener: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotListener = RSA1ShotListener(
  Lexicon(3, 2, Vector(1.0, 0.0, 0.0, 1.0, 1.0, 1.0)),
  0
)
speaker: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotSpeaker = RSA1ShotSpeaker(
  Lexicon(3, 2, Vector(1.0, 0.0, 0.0, 1.0, 1.0, 1.0)),
  0
)
intention: Int = 0
signal: com.markblokpoel.lanag.core.ContentSignal = ContentSignal(Some(0))
inferredIntention: com.markblokpoel.lanag.core.ReferentialIntention = ReferentialIntention(
  Some(0)
)

With two zero-order communicators, there is more chance for misunderstanding. If you run the example multiple times you can observe this.

You can again try to play around with the parameters:

1. Try setting the order to 1 and see what happens. Can both agents successfully disambiguate their lexicons?

2. Try setting the order to 2 or 4, does this help over and above 1st order?

## 4. Interaction
The simulation framework provide an architecture for two agents to communicate. Below is example code how to set up such an interaction, simulate it and print the data. There is some redundancy with the code above for completeness sake of the example.

First we create two agents, each with their own lexicon. Note that the example lexicons are now different, you can play around with them too. Add more signals, more referents, change the relationships and see how this affects the interaction.

In [15]:
val lexicon1 = Lexicon(
    Vector[Vector[Double]](
        Vector(1,0),
        Vector(0,1),
        Vector(1,1)
))
val friend1 = new RSA1ShotAgent(lexicon1, order = 0)

val lexicon2 = Lexicon(
    Vector[Vector[Double]](
        Vector(1,0),
        Vector(1,1),
        Vector(0,1)
))
val friend2 = new RSA1ShotAgent(lexicon2, order = 0)

lexicon1: Lexicon = Lexicon(3, 2, Vector(1.0, 0.0, 0.0, 1.0, 1.0, 1.0))
friend1: RSA1ShotAgent = Agent with order 0
1.0	0.0	
0.0	1.0	
1.0	1.0	

lexicon2: Lexicon = Lexicon(3, 2, Vector(1.0, 0.0, 1.0, 1.0, 0.0, 1.0))
friend2: RSA1ShotAgent = Agent with order 0
1.0	0.0	
1.0	1.0	
0.0	1.0	


With these two agents, we can set up a repeated 1-shot interaction between them. Here, they will take turns as speaker and listener where they will try to communicate a randomly selected intention each turn. Running the interaction will return a dataset that contains information regarding the interaction between the two agents. The more complex code below is just to extract the data for display. For more information on the contents of the dataset, we refer to the API documentation on [`RSA1InteractionData`](https://markblokpoel.github.io/lanag-ambiguityhelps/latest/api/com/markblokpoel/lanag/ambiguityhelps/RSA1ShotDataStructures$$RSA1InteractionData.html).

In [16]:
val interaction = new RSA1ShotInteraction(friend1.withOrder(0), friend2.withOrder(0), maxTurns=10)
val data = interaction.runAndCollectData

Markdown(s"""
### Simulation results

|Variable|Value|
|---|---|
|`friend1`'s ambiguity|${data.agent1AmbiguityMean}|
|`friend2`'s ambiguity|${data.agent2AmbiguityMean}|
|`friend1`'s order|${data.agent1Order}|
|`friend2`'s order|${data.agent2Order}|
|asymmetry | ${friend1.originalLexicon.asymmetryWith(friend2.originalLexicon)}
|mean success|${data.interaction.count(d => d.success)/data.interaction.length.toDouble*100}%|
|number of rounds|${data.interaction.length}|
""")

Markdown(s"""
### Inferred lexicons, even rounds

`friend1` as Speaker

${lex2md(interaction.agent1AsSpeaker.inferredLexicon)} 

`friend2` as Listener

${lex2md(interaction.agent2AsListener.inferredLexicon)}

### Inferred lexicons, odd rounds

`friend2` as Speaker

${lex2md(interaction.agent2AsSpeaker.inferredLexicon)}

`friend1` as Listener

${lex2md(interaction.agent1AsListener.inferredLexicon)}
""")


### Simulation results

|Variable|Value|
|---|---|
|`friend1`'s ambiguity|1.3333333333333333|
|`friend2`'s ambiguity|1.3333333333333333|
|`friend1`'s order|0|
|`friend2`'s order|0|
|asymmetry | 0.3333333333333333
|mean success|90.0%|
|number of rounds|10|



### Inferred lexicons, even rounds

`friend1` as Speaker

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|0.5|0.0|
|$S_1$|0.0|0.5|
|$S_2$|0.5|0.5|
 

`friend2` as Listener

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|1.0|0.0|
|$S_1$|0.5|0.5|
|$S_2$|0.0|1.0|


### Inferred lexicons, odd rounds

`friend2` as Speaker

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|0.5|0.0|
|$S_1$|0.5|0.5|
|$S_2$|0.0|0.5|


`friend1` as Listener

|&nbsp;|$C_0$|$C_1$|
|--|--|--|
|$S_0$|1.0|0.0|
|$S_1$|0.0|1.0|
|$S_2$|0.5|0.5|



interaction: RSA1ShotInteraction = RSA1ShotInteraction(
  Agent with order 0
1.0	0.0	
0.0	1.0	
1.0	1.0	
,
  Agent with order 0
1.0	0.0	
1.0	1.0	
0.0	1.0	
,
  RSA1OriginData(NaN, NaN, NaN, NaN),
  1L,
  10
)
data: com.markblokpoel.lanag.ambiguityhelps.RSA1ShotDataStructures.RSA1InteractionData = RSA1InteractionData(
  1L,
  0,
  0,
  1.3333333333333333,
  0.2222222222222222,
  1.3333333333333333,
  0.2222222222222222,
  0.3333333333333333,
  RSA1OriginData(NaN, NaN, NaN, NaN),
  List(
    RSA1TurnData(0, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
    RSA1TurnData(1, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
    RSA1TurnData(2, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
    RSA1TurnData(3, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
    RSA1TurnData(4, false, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
    RSA1TurnData(5, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
    RSA1TurnData(6, true, SpeakerData(Some(1.0)), List